In [86]:
pip install numpy==1.19.5

Importing the packages

In [87]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import csv
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.convolutional import Conv1D
from keras.models import Sequential
import tqdm
import sklearn.metrics
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dense
from keras.metrics import Recall, Precision
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers


Import packages

In [88]:
dtfrm = []

data_spm = open('/content/spambase_csv.csv')
reader = csv.reader(data_spm)
next(reader, None)

for row in reader:
    dtfrm.append(row)
data_spm.close()

X = [x[:-1] for x in dtfrm]
y = [x[-1] for x in dtfrm] 

In [89]:
t = Tokenizer()
t.fit_on_texts(X)

X = t.texts_to_sequences(X)

X = np.array(X)
y = np.array(y)

X = pad_sequences(X, maxlen=100)

In [90]:
sq_lt = 100 
emsize = 100  
test_size = 0.25 

BATCH_SIZE = 64
EPOCHS = 10 

lblconv = {"ham": 0, "spam": 1}
int2label = {0: "ham", 1: "spam"}

In [91]:
X = np.array(X)
y = np.array(y)
X = pad_sequences(X, maxlen=sq_lt)

In [93]:
y = to_categorical(y)

split and shuffle

In [94]:
test_size = 0.25

trainX, testX, trainY, testY = train_test_split(X, y, test_size=test_size, random_state=7)
s
print("trainX.shape:", trainX.shape)
print("testX.shape:", testX.shape)
print("trainY.shape:", trainY.shape)
print("testY.shape:", testY.shape)

trainX.shape: (3450, 100)
testX.shape: (1151, 100)
trainY.shape: (3450, 2)
testY.shape: (1151, 2)


In [95]:
def embd_vec(dim=100):
  embd_indx = {}

  with open(f"/content/glove.6B.100d.txt", encoding='utf8') as f:
        for line in tqdm.tqdm(f, "Reading GloVe"):
            values = line.split()
            word = values[0]
            vectors = np.asarray(values[1:], dtype='float32')
            embd_indx[word] = vectors

  word_index = t.word_index

  embd_mtx = np.zeros((len(word_index)+1, 100))
  for word, i in word_index.items():
    embedding_vector = embd_indx.get(word)
    if embedding_vector is not None:
      embd_mtx[i] = embedding_vector
  return embd_mtx

In [96]:
embd_mtx = get_embedding_vectors()
model = Sequential()
model.add(Embedding(4513,
              EMBEDDING_SIZE,
              weights=[embd_mtx],
              trainable=False,
              input_length=SEQUENCE_LENGTH))

model.add(LSTM(128, recurrent_dropout=0.2))
model.add(Dropout(0.3))
model.add(Dense(2, activation="softmax"))

Reading GloVe: 400000it [00:19, 20369.82it/s]


In [97]:
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 100, 100)          451300    
                                                                 
 lstm_5 (LSTM)               (None, 128)               117248    
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 2)                 258       
                                                                 
Total params: 568,806
Trainable params: 117,506
Non-trainable params: 451,300
_________________________________________________________________
None


In [98]:
optimum=optimizers.Adam(clipvalue=0.5)
model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])

In [99]:
import time

mdl_chpt = ModelCheckpoint("results/spam_classifier_{val_loss:.2f}.h5", save_best_only=True,
                                    verbose=1)

tensorboard = TensorBoard(f"logs/spam_classifier_{time.time()}")

model.fit(trainX, trainY, validation_data=(testX, testY),
          batch_size=BATCH_SIZE, epochs=EPOCHS,
          callbacks=[tensorboard, mdl_chpt],
          verbose=1)

Epoch 1/10
54/54 [==============================] - ETA: 0s - loss: 0.4916 - accuracy: 0.7603
Epoch 1: val_loss improved from inf to 0.37760, saving model to results/spam_classifier_0.38.h5
54/54 [==============================] - 31s 478ms/step - loss: 0.4916 - accuracy: 0.7603 - val_loss: 0.3776 - val_accuracy: 0.8271
Epoch 2/10
54/54 [==============================] - ETA: 0s - loss: 0.3567 - accuracy: 0.8490
Epoch 2: val_loss did not improve from 0.37760
54/54 [==============================] - 20s 371ms/step - loss: 0.3567 - accuracy: 0.8490 - val_loss: 0.4402 - val_accuracy: 0.7976
Epoch 3/10
54/54 [==============================] - ETA: 0s - loss: 0.3108 - accuracy: 0.8751
Epoch 3: val_loss improved from 0.37760 to 0.31120, saving model to results/spam_classifier_0.31.h5
54/54 [==============================] - 18s 321ms/step - loss: 0.3108 - accuracy: 0.8751 - val_loss: 0.3112 - val_accuracy: 0.8801
Epoch 4/10
54/54 [==============================] - ETA: 0s - loss: 0.2991 - ac

In [100]:

output = model.evaluate(testX, testY)

loss = output[0]
accuracy = output[1]

print(f"[+] Accuracy: {accuracy*100:.2f}%")

36/36 [==============================] - 1s 34ms/step - loss: 0.3751 - accuracy: 0.8349
[+] Accuracy: 83.49%
